In [7]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
# os.chdir('02514-Deep-Learning-In-Computer-Vision/')
import numpy as np
import glob
import PIL.Image as Image

# pip install torchsummary
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, random_split
import torchvision.transforms as transforms
from torchvision import models
from torchsummary import summary
import torch.optim as optim
from time import time
from dataloader import Lesion_Data

import matplotlib.pyplot as plt
from IPython.display import clear_output

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Running on ", device)
torch.cuda.empty_cache()

Running on  cuda


## Load the dataset

In [2]:
size = 128
batch_size = 6

dataset = Lesion_Data(train_transform_size=size, test_transform_size=size)
trainset, testset, validationset = dataset.get_datasets()

train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=1)
test_loader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=1)
validation_loader = DataLoader(validationset, batch_size=batch_size, shuffle=False, num_workers=1)
print('Loaded %d training images' % len(trainset))
print('Loaded %d test images' % len(testset))
print('Loaded %d validation images' % len(validationset))

Loaded 128 training images
Loaded 40 test images
Loaded 32 validation images


In [5]:
plt.rcParams['figure.figsize'] = [18, 6]

images, labels = next(iter(train_loader))

for i in range(6):
    plt.subplot(2, 6, i+1)
    plt.imshow(np.swapaxes(np.swapaxes(images[i], 0, 2), 0, 1))

    plt.subplot(2, 6, i+7)
    plt.imshow(labels[i].squeeze())
plt.show()

In [5]:
! nvidia-smi

zsh:1: command not found: nvidia-smi


In [6]:
torch.cuda.device('cuda:0')

In [3]:
def train(model, opt, loss_fn, epochs, train_loader, test_loader):
    X_test, Y_test = next(iter(test_loader))

    for epoch in range(epochs):
        tic = time()
        print('* Epoch %d/%d' % (epoch+1, epochs))

        avg_loss = 0
        model.train()  # train mode
        for X_batch, Y_batch in train_loader:
            X_batch = X_batch.to(device)
            Y_batch = Y_batch.to(device)

            # set parameter gradients to zero
            opt.zero_grad()

            # forward
            Y_pred = model(X_batch)
            # loss = loss_fn(Y_batch, torch.argmax(Y_pred, dim = 1))  # forward-pass
            loss = loss_fn(Y_batch, Y_pred)  # forward-pass
            loss.backward()  # backward-pass
            opt.step()  # update weights

            # calculate metrics to show the user
            avg_loss += loss / len(train_loader)
        toc = time()
        print(' - loss: %f' % avg_loss)

        # show intermediate results
        model.eval()  # testing mode
        Y_hat = F.sigmoid(model(X_test.to(device))).detach().cpu()
        clear_output(wait=True)
        for k in range(4):
            plt.subplot(2, 6, k+1)
            plt.imshow(np.rollaxis(X_test[k].numpy(), 0, 3), cmap='gray')
            plt.title('Real')
            plt.axis('off')

            plt.subplot(2, 6, k+7)
            plt.imshow(Y_hat[k, 0], cmap='gray')
            plt.title('Output')
            plt.axis('off')
        plt.suptitle('%d / %d - loss: %f' % (epoch+1, epochs, avg_loss))
        plt.show()

In [4]:
def bce_loss(y_real, y_pred, clip = False):
    loss = nn.BCELoss()
    #pos_weight = torch.ones([size])
    #loss = nn.BCEWithLogitsLoss(pos_weight)
    return loss((y_pred), y_real)

In [9]:
def predict(model, data):
    model.eval()  # testing mode
    Y_pred = [F.sigmoid(model(X_batch.to(device))) for X_batch, _ in data]
    return np.array(Y_pred)

# SimpleCNN

In [5]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.convolutional = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2), # 128 -> 64
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1), 
            nn.ReLU(),
            nn.MaxPool2d(2,2), # 64 -> 32
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
        
        )
        
        self.fully_connected = nn.Sequential(
            nn.Linear(64*32*32, 128),
            nn.ReLU(),
            nn.Linear(128, 128*128), # output with 128x128 dimensions. 
        )

    def forward(self, x):
        x = self.convolutional(x)
        x = x.view(x.size(0), -1)
        x = self.fully_connected(x)
        x = x.view(x.size(0), 1, 128, 128)  # Reshape output to match target size
        return x

In [8]:
# Leasion | Not leasion data
model = SimpleCNN().to(device)
summary(model, (3, 128, 128))
train(model, optim.Adam(model.parameters()), bce_loss, 50, train_loader, test_loader)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 128, 128]             448
              ReLU-2         [-1, 16, 128, 128]               0
         MaxPool2d-3           [-1, 16, 64, 64]               0
            Conv2d-4           [-1, 32, 64, 64]           4,640
              ReLU-5           [-1, 32, 64, 64]               0
         MaxPool2d-6           [-1, 32, 32, 32]               0
            Conv2d-7           [-1, 64, 32, 32]          18,496
              ReLU-8           [-1, 64, 32, 32]               0
            Linear-9                  [-1, 128]       8,388,736
             ReLU-10                  [-1, 128]               0
           Linear-11                [-1, 16384]       2,113,536
Total params: 10,525,856
Trainable params: 10,525,856
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.19
Fo

../aten/src/ATen/native/cuda/Loss.cu:92: operator(): block: [119,0,0], thread: [98,0,0] Assertion `input_val >= zero && input_val <= one` failed.
../aten/src/ATen/native/cuda/Loss.cu:92: operator(): block: [119,0,0], thread: [99,0,0] Assertion `input_val >= zero && input_val <= one` failed.
../aten/src/ATen/native/cuda/Loss.cu:92: operator(): block: [119,0,0], thread: [102,0,0] Assertion `input_val >= zero && input_val <= one` failed.
../aten/src/ATen/native/cuda/Loss.cu:92: operator(): block: [119,0,0], thread: [103,0,0] Assertion `input_val >= zero && input_val <= one` failed.
../aten/src/ATen/native/cuda/Loss.cu:92: operator(): block: [119,0,0], thread: [111,0,0] Assertion `input_val >= zero && input_val <= one` failed.
../aten/src/ATen/native/cuda/Loss.cu:92: operator(): block: [119,0,0], thread: [112,0,0] Assertion `input_val >= zero && input_val <= one` failed.
../aten/src/ATen/native/cuda/Loss.cu:92: operator(): block: [119,0,0], thread: [116,0,0] Assertion `input_val >= zero &&

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [ ]:
class SimpleCNN2(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.convolutional = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2), # 128 -> 64
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1), 
            nn.ReLU(),
            nn.MaxPool2d(2,2), # 64 -> 32
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2), # 32 -> 16
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        
        )
        
        self.fully_connected = nn.Sequential(
            nn.Linear(128*16*16, 128),
            nn.ReLU(),
            nn.Linear(128, 128*128), # output with 128x128 dimensions. 
            
        )

    def forward(self, x):
        x = self.convolutional(x)
        x = x.view(x.size(0), -1)
        x = self.fully_connected(x)
        x = x.view(x.size(0), 1, 128, 128)  # Reshape output to match target size
        return x

In [ ]:
# Leasion | Not leasion data
model = SimpleCNN2().to(device)
summary(model, (3, 128, 128))
train(model, optim.Adam(model.parameters()), bce_loss, 50, train_loader, test_loader)

# With sigmoid binaries

In [10]:
class SimpleCNNB(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.convolutional = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2), # 128 -> 64
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1), 
            nn.ReLU(),
            nn.MaxPool2d(2,2), # 64 -> 32
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
        
        )
        
        self.fully_connected = nn.Sequential(
            nn.Linear(64*32*32, 128),
            nn.ReLU(),
            nn.Linear(128, 128*128), # output with 128x128 dimensions.
        )

    def forward(self, x):
        x = self.convolutional(x)
        x = x.view(x.size(0), -1)
        x = self.fully_connected(x)
        x = x.view(x.size(0), 1, 128,128)  # Reshape output to match target sizez
        x = torch.sigmoid(x) # Apply element wise sigmoid
        x = (x > 0.5).float()  # Thresholding with a given threshold
        
         # Enable gradient calculation for the thresholded tensor
        x.requires_grad_()
        return x

In [ ]:
# Leasion | Not leasion data
model = SimpleCNNB().to(device)
summary(model, (3, 128, 128))
train(model, optim.Adam(model.parameters()), bce_loss, 50, train_loader, test_loader)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 128, 128]             448
              ReLU-2         [-1, 16, 128, 128]               0
         MaxPool2d-3           [-1, 16, 64, 64]               0
            Conv2d-4           [-1, 32, 64, 64]           4,640
              ReLU-5           [-1, 32, 64, 64]               0
         MaxPool2d-6           [-1, 32, 32, 32]               0
            Conv2d-7           [-1, 64, 32, 32]          18,496
              ReLU-8           [-1, 64, 32, 32]               0
            Linear-9                  [-1, 128]       8,388,736
             ReLU-10                  [-1, 128]               0
           Linear-11                [-1, 16384]       2,113,536
Total params: 10,525,856
Trainable params: 10,525,856
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.19
Fo

In [ ]:
class SimpleCNN2B(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.convolutional = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2), # 128 -> 64
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1), 
            nn.ReLU(),
            nn.MaxPool2d(2,2), # 64 -> 32
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2), # 32 -> 16
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        
        )
        
        self.fully_connected = nn.Sequential(
            nn.Linear(128*16*16, 128),
            nn.ReLU(),
            nn.Linear(128, 128*128), # output with 128x128 dimensions. 
            nn.Sigmoid()
            
        )

    def forward(self, x):
        x = self.convolutional(x)
        x = x.view(x.size(0), -1)
        x = self.fully_connected(x)
        x = x.view(x.size(0), 1, 128, 128)  # Reshape output to match target size
        return x

In [ ]:
# Leasion | Not leasion data
model = SimpleCNN2B().to(device)
summary(model, (3, 128, 128))
train(model, optim.Adam(model.parameters()), bce_loss, 50, train_loader, test_loader)